<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Aerial_Photograph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import folium
import time
from IPython.display import Image
import geemap


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

**Import image to GEE**
The first step is to import the image from GEE

In [ ]:
# Define the image
image = ee.Image('projects/mapwaps-sabiecroc/assets/2530BB_24_2018_1321_RGB_RECT')

Adding indices

In [ ]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('b2'),
    'RED': image.select('b1'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('b3'),
    'Red': image.select('b1'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo(), 'image_bands')